# <a id='toc1_'></a>[Part 1. Notebook for Network and Topological Analysis in Neuroscience](#toc0_)

Authors: Isaac Sebenius (Inspired heavily from the work by Eduarda Centeno & Fernando Santos), see [Acknowledgements](#acknowledgements) 

**Table of contents**<a id='toc0_'></a>    
- [Part 1. Notebook for Network and Topological Analysis in Neuroscience](#toc1_)    
    - [ **1.** Imports](#toc1_1_1_)    
      - [Let's start with the necessary packages for the following computations:](#toc1_1_1_1_)    
    - [**2.** Importing data & exploring connectivity matrices](#toc1_1_2_)    
      - [Now we will start working with the brain data.](#toc1_1_2_1_)    
        - [**Let's look at what these group-averaged networks look like, using a standard heatmap!**](#toc1_1_2_1_1_)    
    - [How do these networks relate to each other? What do you expect the correlation between these networks to be?](#toc1_1_3_)    
    - [ What's the takeaway here? What are other ways we could go about studying the relationship between structure and function in the brain?](#toc1_1_4_)    
        - [**Key point:**](#toc1_1_4_1_1_)    
        - [**Key point:**](#toc1_1_4_1_2_)    
    - [**3.**  Graph Theory](#toc1_1_5_)    
      - [From now, we will start working with some standard Graph Theory metrics.](#toc1_1_5_1_)    
        - [**Key point:**](#toc1_1_5_1_1_)    
        - [We will start by creating the graph and removing its self-loops (i.e., a connection of a node with itself). For illustrative purposes, we will focus on the group-level fMRI conenctivity graph.](#toc1_1_5_1_2_)    
        - [Now, we compute the network's density.](#toc1_1_5_1_3_)    
        - [Now, we compute the nodal degree/strength.](#toc1_1_5_1_4_)    
        - [Next, we will compute the centralities!](#toc1_1_5_1_5_)    
        - [Now, let's move on to the Path Length!](#toc1_1_5_1_6_)    
- [Now, go back and redo this past section using graphs at different levels of sparsity. How do the results change?](#toc2_)    
- [Explore the relationships between differnet node-level graph features. How do they compare to each other? What might this tell us?](#toc3_)    
        - [Now, modularity, assortativity, clustering coefficient and the minimum spanning tree!](#toc3_1_1_1_1_)    
      - [Data Visualisation & Graph Theory](#toc3_1_1_2_)    
- [Obviously that is WAY too much information. Let's visualise the Minimum Spanning Tree instead.](#toc4_)    
  - [**4.** Have some fun with data analysis of brain networks!!](#toc4_1_)    
  - [**5.** References](#toc4_2_)    
  - [**6.** Acknowledgements](#toc4_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='imports'></a>
### <a id='toc1_1_1_'></a>[ **1.** Imports](#toc0_)
#### <a id='toc1_1_1_1_'></a>[Let's start with the necessary packages for the following computations:](#toc0_)

In [ ]:
# Basic data manipulation and visualisation libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats

# Network Libraries
import networkx as nx

# CANNOT CONDA INSTALL BELOW?
from nxviz import CircosPlot, circos
import community


# Magic command to change plotting backend
#%matplotlib qt

# Magic command to load watermark
#%load_ext watermark

In [ ]:
# Possibility to stop warnings
#import warnings
#warnings.filterwarnings('ignore') 

<a id='importing-data'></a>
### <a id='toc1_1_2_'></a>[**2.** Importing data & exploring connectivity matrices](#toc0_)
#### <a id='toc1_1_2_1_'></a>[Now we will start working with the brain data.](#toc0_)
Here, we will try to cover both computation and some theoretical background/key points on each section. 
Our first step will be on how to import the matrix data:

In [ ]:
import pickle
import os
os.chdir('/imaging/henson/users/rh01/Methods/Cognestic-Stats/CognesticNetworks')

#Load individual data for 100 subjects from the HCP dataset
fmri_connectivity = pickle.load(open('Simplified/FCs.HCP.pkl','rb'))
mind_connectivity = pickle.load(open('Simplified/MIND.HCP.pkl','rb'))
dti_connectivity = pickle.load(open('Simplified/DTI.HCP.pkl','rb'))

mask = np.ones(fmri_connectivity[0].shape)
mask[np.diag_indices(360)] = np.nan

#Compute group-averaged networks 
mean_fmri_connectivity = np.mean(fmri_connectivity, axis = 0)*mask
mean_mind_connectivity = np.mean(mind_connectivity, axis = 0)*mask
mean_dti_connectivity = np.mean(dti_connectivity, axis = 0)*mask


In [ ]:
region_information = pd.read_csv('HCP-coordinates.csv')

In [ ]:
# Obtaining name of areas according to matching file
lineList = list(region_information['regionName'].values.flatten())

# Obtaining a random list of numbers to simulate subnetworks -- THESE NUMBERS DO NOT CORRESPOND TO ANY REAL CLASSIFICATION
sublist = np.array(np.loadtxt('HCP_Yeo_symmetric.txt'))

cmap_dict = dict(zip(list(range(8)), sns.color_palette("tab10", n_colors = 8)))
# Obtaining a random list of colors that will match the random subnetwork classification for further graphs -- THESE COLORNAMES DO NOT CORRESPOND TO ANY REAL CLASSIFICATION
colorlist = np.array([cmap_dict[x] for x in sublist])

# Obtaining a random list of colors (in numbers) that will match the random subnetwork classification for further graphs -- THESE NUMBERS DO NOT CORRESPOND TO ANY REAL CLASSIFICATION
colornumbs = np.array(colorlist.copy())#np.genfromtxt('./subnet_colors_number.txt')

##################################################################################################
##### <a id='toc1_1_2_1_1_'></a>[**Let's look at what these group-averaged networks look like, using a standard heatmap!**](#toc0_)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5))

sns.heatmap(mean_fmri_connectivity, ax = ax[0], square = True, cbar_kws = {"shrink": 0.75})
sns.heatmap(mean_mind_connectivity, ax = ax[1], square = True, cbar_kws = {"shrink": 0.75})
sns.heatmap(mean_dti_connectivity, ax = ax[2], square = True, cbar_kws = {"shrink": 0.75})

ax[0].set_title('Mean Functional Connectivity')
ax[1].set_title('Mean DTI Connectivity')
ax[2].set_title('Mean MIND Connectivity')
plt.show()

##################################################################################################

### <a id='toc1_1_3_'></a>[How do these networks relate to each other? What do you expect the correlation between these networks to be?](#toc0_)

In [ ]:
print('DTI - Functional connectivity edgewise correlation:', str(round(stats.pearsonr(mean_fmri_connectivity[np.triu_indices(360, k = 1)], mean_dti_connectivity[np.triu_indices(360, k = 1)])[0], 2)))
print('MIND - Functional conenctivity edgewise correlation:', str(round(stats.pearsonr(mean_fmri_connectivity[np.triu_indices(360, k = 1)], mean_mind_connectivity[np.triu_indices(360, k = 1)])[0], 2)))
print('DTI - MIND edgewise correlation:', str(round(stats.pearsonr(mean_mind_connectivity[np.triu_indices(360, k = 1)], mean_dti_connectivity[np.triu_indices(360, k = 1)])[0], 2)))

### <a id='toc1_1_4_'></a>[ What's the takeaway here? What are other ways we could go about studying the relationship between structure and function in the brain?](#toc0_)

##################################################################################################

##### <a id='toc1_1_4_1_1_'></a>[**Key point:**](#toc0_)
When working with network analysis in brain data, a couple of crucial decisions have to be made.
For example, one can decide to use all network connections - including low-weight links (sometimes considered spurious connections), or establish an arbitrary threshold and keep only links above a specific correlation value. This step can be done in different ways, based solely on the correlation threshold (as done here), or based on network density (i.e., you keep only the 20% strongest correlations). 
If using an arbitrary threshold, it is also possible to define if the resulting matrix will be weighted (i.e., keeping the edges' weight), or unweighted (binarised matrices).

Another point of discussion is how to deal with negative weights in weighted networks. Here, we have chosen to proceed by thresholding for  the top weighted connections, ignoring negative/low connections.

*We strongly suggest Reference [3] for a deeper understanding on all these decisions.*

Figure 1 provides a schematic summary of the types of networks:

![matrices.jpg](attachment:matrices.jpg)

Figure 1. Types of networks. (A) A binary directed graph. (B) Binary, undirected graph. In binary graphs, the presence of a connection is signified by a 1 or 0 otherwise. (C) A representation of graph F as a network of brain areas. (D) A weighted, directed graph. (F) A weighted, undirected graph. In a weighted graph, the strength of the connections is represented by a number [0,1]. (G) A connectivity matrix of C and F. Source: Part of the image was obtained from [Smart Servier Medical Art](https://smart.servier.com/).

##### <a id='toc1_1_4_1_2_'></a>[**Key point:**](#toc0_)
When working with fMRI brain network data, it is useful to generate some plots (e.g., the heatmaps above for matrix visualisation, and distribution plots of edge weights) to facilitate data comprehension and flag potential artefacts. In brain networks, we expect mostly weak edges and a smaller proportion of strong ones. Is that what we see?

In [ ]:
# Weight distribution plot

def MinMax(X):
    X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
    #X_scaled = X_std * (max - min) + min
    return X_scaled

centiled_fmri_edges = MinMax(mean_fmri_connectivity[np.triu_indices(360, k =1)])
centiled_dti_edges = MinMax(mean_dti_connectivity[np.triu_indices(360, k =1)])
centiled_mind_edges = MinMax(mean_mind_connectivity[np.triu_indices(360, k =1)])

fig, ax = plt.subplots(1, 3, figsize = (15, 5))

sns.histplot(centiled_fmri_edges, ax = ax[0])
sns.histplot(centiled_mind_edges, ax = ax[1])
sns.histplot(centiled_dti_edges, ax = ax[2])

ax[0].set_xlabel('Scaled functional connectivity')
ax[1].set_xlabel('Scaled MIND connectivity')
ax[2].set_xlabel('Scaled DTI connectivity')

plt.show()

<a id='graph-theory'></a>
### <a id='toc1_1_5_'></a>[**3.**  Graph Theory](#toc0_)
#### <a id='toc1_1_5_1_'></a>[From now, we will start working with some standard Graph Theory metrics.](#toc0_)


The metrics that we will cover here are:
- Density
- Degree/Strength
- Centrality: Eigenvector, Betweenness, Closeness, Degree, Page Rank
- Path length
- Modularity
- Assortativity
- Clustering coefficient

##### <a id='toc1_1_5_1_1_'></a>[**Key point:**](#toc0_)
Each of these metrics has its requisites for computation. For example, it is not possible to accurately compute closeness centrality and the average shortest path for fragmented networks (i.e., there are subsets of disconnected nodes). Therefore, keep that in mind when thinking about thresholding a matrix.

Figure 2 provides a summary of some graph-theoretical metrics:

![GT.jpg](attachment:GT.jpg)
Figure 2. Graph theoretical metrics. (A) A representation of a graph indicating centralities. (B) Representation of modularity and clustering coefficient. (C) The shortest path between vertices A and B. (D) The minimum spanning tree.

##### <a id='toc1_1_5_1_2_'></a>[We will start by creating the graph and removing its self-loops (i.e., a connection of a node with itself). For illustrative purposes, we will focus on the group-level fMRI conenctivity graph.](#toc0_)

In [ ]:
# Creating a graph
centiled_fc = np.zeros(mean_fmri_connectivity.shape)
centiled_fc[np.triu_indices(360, k = 1)] = centiled_fmri_edges
centiled_fc += centiled_fc.T

matrix = centiled_fc.copy()#mean_fmri_connectivity.copy()
G = nx.from_numpy_array(matrix)

# Removing self-loops
G.remove_edges_from(list(nx.selfloop_edges(G)))

##### <a id='toc1_1_5_1_3_'></a>[Now, we compute the network's density.](#toc0_)


<u>Definition</u>:
 A graph's density is the ratio between the number of edges and the total number of possible edges. 

Clearly, in all-to-all connected graphs, the density will be maximal (or 1), whereas for a graph without edges it will be 0. Here, just for the sake of demonstration, we will compute the density of different states of the network to show how density changes.

In [ ]:
#print(nx.density.__doc__)

# Create graphs for comparison
matrix2 = matrix.copy()
matrix3 = matrix.copy()

# Create sparser graphs
matrix2[matrix2<=0.50] = 0
matrix3[matrix3<=0.75] = 0

st50G = nx.from_numpy_array(matrix2)
st25G = nx.from_numpy_array(matrix3)

st50G.remove_edges_from(list(nx.selfloop_edges(st50G)))
st25G.remove_edges_from(list(nx.selfloop_edges(st25G)))

# Compute densities
alltoall = nx.density(G)
st50 = nx.density(st50G)
st25 = nx.density(st25G)

names = ['All-To-All', '> 0.5', '> 0.75']
values = [alltoall, st50, st25]

dict(zip(names, values))

##### <a id='toc1_1_5_1_4_'></a>[Now, we compute the nodal degree/strength.](#toc0_)

<u>Definition</u>: In undirected weighted networks the node strength can be computed as the sum of the connectivity weights of the edges attached to each node. It is a primary metric to identify how important is a node in the graph. 
It is possible to apply a normalization (divide the weights by 1/N-1) to make the output value more intuitive. (Reference [3] pg. 119)  

In degree computation, it is also common to compute the mean degree of the network, which is the sum of node degrees divides by the total number of nodes.

In [ ]:
# Computation of nodal degree/strength
#print(nx.degree.__doc__)

strength = G.degree(weight='weight')
strengths = {node: val for (node, val) in strength}
nx.set_node_attributes(G, dict(strength), 'strength') # Add as nodal attribute

# Normalized node strength values 1/N-1
normstrenghts = {node: val * 1/(len(G.nodes)-1) for (node, val) in strength}
nx.set_node_attributes(G, normstrenghts, 'strengthnorm') # Add as nodal attribute

# Computing the mean degree of the network
normstrengthlist = np.array([val * 1/(len(G.nodes)-1) for (node, val) in strength])
mean_degree = np.sum(normstrengthlist)/len(G.nodes)

import warnings # pymer4 needs to be updated to python3.1, so suppress warning to do with dataframe

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning) # suppress warning about old version of distplot
    sns.distplot(list(normstrengthlist), kde=True, norm_hist=False)
plt.xlabel('Degree Values');
plt.ylabel('Counts');

##### <a id='toc1_1_5_1_5_'></a>[Next, we will compute the centralities!](#toc0_)

Centralities are frequently used to understand which nodes occupy critical positions in the network.

Remember: 
- Degree Centrality: The degree centrality for a node **v** is the fraction of nodes it is connected to. This metric is the same as node degree, so it will not be computed again. (NetworkX Documentation [4])

- Closeness Centrality: In weighted graphs, the closeness centrality of a node __v__ is the reciprocal of the sum of the shortest weighted path distances from **v** to all *N-1* other nodes. An important thing to think about this metric is that a node with many low weight edges can have the same centrality as a node with only a few high-weighted edges. (NetworkX Documentation, Reference [3] - Chapter 5)

- Betweenness Centrality: Betweenness centrality of a node **v** is the sum of the fraction of all-pairs shortest paths that pass through __v__. (NetworkX Documentation [4]) 

- Eigenvector Centrality: Eigenvector centrality computes the centrality for a node based on its neighbours' centrality. It takes into account not only quantity (e.g., degree centrality) but also quality. If a node is linked to many nodes that also display a high degree, that node will have high eigenvector centrality. (NetworkX Documentation)

- Page Rank: PageRank computes a ranking of the nodes in the graph G based on the incoming links' structure. (NetworkX Documentation [4])

In [ ]:
# Closeness centrality
#print(nx.closeness_centrality.__doc__)

# The function accepts a argument 'distance' that, in correlation-based networks, must be seen as the inverse ... 
# of the weight value. Thus, a high correlation value (e.g., 0.8) means a shorter distance (i.e., 0.2).
G_distance_dict = {(e1, e2): 1 / abs(weight) for e1, e2, weight in G.edges(data='weight')}

# Then add them as attributes to the graph edges
nx.set_edge_attributes(G, G_distance_dict, 'distance')

# Computation of Closeness Centrality
closeness = nx.closeness_centrality(G, distance='distance')

# Now we add the closeness centrality value as an attribute to the nodes
nx.set_node_attributes(G, closeness, 'closecent')

# Visualise  values directly
#print(closeness)

# Closeness Centrality Histogram
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning) # suppress warning about old version of distplot
    sns.distplot(list(closeness.values()), kde=True, norm_hist=False)
plt.xlabel('Centrality Values');
plt.ylabel('Counts');

In [ ]:
# Betweenness centrality:
#print(nx.betweenness_centrality.__doc__)
betweenness = nx.betweenness_centrality(G, weight='distance', normalized=True) 
                                                                
# Now we add the it as an attribute to the nodes
#nx.set_node_attributes(G, betweenness, 'bc')

# Visualise  values directly
#print(betweenness)

# Betweenness centrality Histogram
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning) # suppress warning about old version of distplot
    sns.distplot(list(betweenness.values()), kde=False, norm_hist=False)
plt.xlabel('Centrality Values');
plt.ylabel('Counts');

In [ ]:
# Eigenvector centrality
#print(nx.eigenvector_centrality.__doc__)
eigen = nx.eigenvector_centrality(G, weight='weight')

# Now we add the it as an attribute to the nodes
nx.set_node_attributes(G, eigen, 'eigen')

# Visualise  values directly
#print(eigen)

# Eigenvector centrality Histogram
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning) # suppress warning about old version of distplot
    sns.distplot(list(eigen.values()), kde=False, norm_hist=False)
plt.xlabel('Centrality Values');
plt.ylabel('Counts');

In [ ]:
# Page Rank
#print(nx.pagerank.__doc__)
pagerank = nx.pagerank(G, weight='weight')

# Add as attribute to nodes
nx.set_node_attributes(G, pagerank, 'pg')

# Visualise values directly
#print(pagerank)

# Page Rank Histogram
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning) # suppress warning about old version of distplot
    sns.distplot(list(pagerank.values()), kde=False, norm_hist=False)
plt.xlabel('Pagerank Values')
plt.ylabel('Counts')

##### <a id='toc1_1_5_1_6_'></a>[Now, let's move on to the Path Length!](#toc0_)

- Shortest Path:  The shortest path (or distance) between two nodes in a graph. In a weighted graph it is obtained by the minimum sum of weights.

- Average Path Length: It is a concept in network topology that is defined as the average number of steps along the shortest paths for all possible pairs of network nodes. It is a measure of the efficiency of information or mass transport on a network.

In [ ]:
# Path Length
#print(nx.shortest_path_length.__doc__)

# This is a versatile version of the ones below in which one can define or not source and target. Remove the hashtag to use this version.
#list(nx.shortest_path_length(G, weight='distance'))

# This one can also be used if defining source and target: 
#print(nx.dijkstra_path_length.__doc__)
nx.dijkstra_path_length(G, source=20, target=25, weight='distance')

# Whereas this one is for all pairs. Remove the hashtag to use this version.
#print(nx.all_pairs_dijkstra_path_length.__doc__)
#list(nx.all_pairs_dijkstra_path_length(G, weight='distance'))

In [ ]:
# Average Path Length or Characteristic Path Length
#print(nx.average_shortest_path_length.__doc__)
nx.average_shortest_path_length(G, weight='distance')

# <a id='toc2_'></a>[Now, go back and redo this past section using graphs at different levels of sparsity. How do the results change?](#toc0_)

# <a id='toc3_'></a>[Explore the relationships between differnet node-level graph features. How do they compare to each other? What might this tell us?](#toc0_)

In [ ]:
###### Code here  ########





#########################

##### <a id='toc3_1_1_1_1_'></a>[Now, modularity, assortativity, clustering coefficient and the minimum spanning tree!](#toc0_)

- Modularity: Modularity compares the number of edges inside a cluster with the expected number of edges that one would find if the network was connected randomly but with the same number of nodes and node degrees. It is used to identify strongly connected subsets, i.e., modules or 'communities'. Here, we will use the Louvain algorithm, as recommended in Reference [3].

- Assortativity: Assortativity measures the similarity of connections in the graph with respect to the node degree. (NetworkX)

- Efficiency: The efficiency of a pair of nodes in a graph is the multiplicative inverse of the shortest path distance between the nodes.  More efficient -> shorter average path between nodes. (NetworkX documentation)

- Clustering coefficient: a measure of the tendency for any two neighbours of a node to be directly connected. According to Networkx's documentation, weighted graphs' clustering coefficient is defined as the geometric average of the subgraph edge weights. (NetworkX, Reference [4])

- Small-worldness: A small world network is characterized by a small average shortest path length, and a large clustering coefficient. Small-worldness is commonly measured with the coefficient sigma or omega. Both coefficients compare the average clustering coefficient and shortest path length of a given graph against the same quantities for an equivalent random or lattice graph. (NetworkX documentation)
 
- Minimum Spanning Tree: it is the backbone of a network, i.e. the minimum set of edges necessary to ensure that paths exist between all nodes. A few main algorithms are used to build the spanning tree, being the Kruskal's algorithm the one used by NetworkX. Briefly, this algorithm ranks the distance of the edges, adds the ones with the smallest distance first, and by adding edge-by-edge, it checks if cycles are formed or not. The algorithm will not add an edge that results in the formation of a cycle.

In [ ]:
# Modularity
#print(community.best_partition.__doc__)
#from community import best_partition
part = community.best_partition(G, weight='weight')

# Visualise values directly
#print(part)

# Check the number of communities
set(part.values()).union()

In [ ]:
#Assortativity
#print(nx.degree_pearson_correlation_coefficient.__doc__)
nx.degree_pearson_correlation_coefficient(G, weight='weight')

In [ ]:
#Efficiency
nx.global_efficiency(st50G)

In [ ]:
#Small worldness (takes ages)
#nx.sigma(st50G, niter=100, nrand=10, seed=None)

In [ ]:
# Clustering Coefficient
#print(nx.clustering.__doc__)
clustering = nx.clustering(G, weight='weight')

# Add as attribute to nodes
nx.set_node_attributes(G, clustering, 'cc')

# Visualise values directly
#print(clustering)

# Clustering Coefficient Histogram
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning) # suppress warning about old version of distplot
    sns.distplot(list(clustering.values()), kde=False, norm_hist=False)
plt.xlabel('Clustering Coefficient Values')
plt.ylabel('Counts')

In [ ]:
# Minimum Spanning Tree
GMST = nx.minimum_spanning_tree(G, weight='distance')

#### <a id='toc3_1_1_2_'></a>[Data Visualisation & Graph Theory](#toc0_)
Under this section we we'll provide a few ideas of how to visualise and present your network.

First, let's get some important attributes about brain area names and subnetworks. These will be used later for graphical visualisation!

In [ ]:
# Function to transform our list of brain areas into a dictionary
def Convert(lst): 
    res_dct = {i : lst[i] for i in range(0, len(lst))} 
    return res_dct

# Add brain areas as attribute of nodes
nx.set_node_attributes(G, Convert(lineList), 'area')

# Add node colors
nx.set_node_attributes(G, Convert(colorlist), 'color')

# Add subnetwork attribute
nx.set_node_attributes(G, Convert(sublist), 'subnet')

# Add node color numbers
#nx.set_node_attributes(G, Convert(colornumbs), 'colornumb')
nx.set_node_attributes(G, Convert(colornumbs), 'colornumb')

Now we will create a standard spring network plot, but this could also be made circular by changing to *draw_circular*.

We defined the edge widths to the power of 2 so that weak weights will have smaller widths.

In [ ]:
# Standard Network graph with nodes in proportion to Graph degrees
plt.figure(figsize=(30,30))
edgewidth = [ d['weight'] for (u,v,d) in G.edges(data=True)]
pos = nx.spring_layout(G, scale=5)
nx.draw(G, pos, with_labels=True, width=np.power(edgewidth, 2), edge_color='grey', node_size=normstrengthlist*20000, 
        labels=Convert(lineList), font_color='black', node_color=colornumbs/10, cmap=plt.cm.Spectral, alpha=0.7, font_size=9)
#plt.savefig('network.jpeg')

# <a id='toc4_'></a>[Obviously that is WAY too much information. Let's visualise the Minimum Spanning Tree instead.](#toc0_)

In [ ]:

plt.figure(figsize=(15,15))
nx.draw(GMST, with_labels=True, alpha=0.7, labels=Convert(lineList), font_size=9)
#nx.draw(GMST, with_labels=True, width=np.power(edgewidth, 0.5), edge_color='grey', node_size=normstrengthlist*200, 
#        labels=Convert(lineList), font_color='black', node_color=colornumbs/10, cmap=plt.cm.Spectral, alpha=0.7, font_size=9)

* Detail! 
For the sake of a less overwhelming plot, we will work with the st50G graph for the CircosPlot.

In [ ]:

nx.set_node_attributes(GMST, dict(GMST.degree(weight='weight')), 'strength')

nx.set_node_attributes(GMST, Convert(lineList), 'area')

nx.set_node_attributes(GMST, Convert(colorlist), 'color')

nx.set_node_attributes(GMST, Convert(sublist), 'subnet')

#edgecolors = {(e1, e2): int((weight+1)**3) for e1, e2, weight in st50G.edges(data='weight')}

# Then add them as attributes to the graph
#nx.set_edge_attributes(st50G, edgecolors, 'edgecolor')

G_distance_dict2 = {(e1, e2): 1 / abs(weight) for e1, e2, weight in GMST.edges(data='weight')}

# Then add them as attributes to the graph
nx.set_edge_attributes(GMST, G_distance_dict2, 'distance')


GMST_GRL = nx.relabel_nodes(GMST, {i: lineList[i] for i in range(len(lineList))})

# CircosPlot
fig, ax = plt.subplots(1,1,figsize = (30,30))
circ = circos(GMST_GRL, edge_alpha_by="weight", node_color_by="subnet")#, figsize=(30,30))#, node_labels=True, node_label_layout='rotation', node_order='subnet',
                 # edge_color='weight', edge_width='weight', node_color='subnet', node_label_color=True, fontsize=10, 
                 # nodeprops={"radius": 2}, group_legend=True, group_label_offset=5)

plt.show()

<a id='Data Analysis on brain networks'></a>

## <a id='toc4_1_'></a>[**4.** Have some fun with data analysis of brain networks!!](#toc0_)

Here, spend about an hour explore relationships between different types of brain networks, and the relationship between different connectivity/network features and phenotypic features such as age and sex. Have some fune with it, be creative – I'll come around and talk to each of you.

If want meta-data on age, sex, etc, for correlations:

In [ ]:
metadata = pd.read_csv('Simplified/metadata.csv')

<a id='references'></a>

## <a id='toc4_2_'></a>[**5.** References](#toc0_)

[1] Brown JA, Rudie JD, Bandrowski A, Van Horn JD, Bookheimer SY. The UCLA multimodal connectivity database: a web-based platform for brain connectivity matrix sharing and analysis. Front Neuroinform. 2012;6:28. doi: 10.3389/fninf.2012.00028.

[2] Biswal BB, Mennes M, Zuo XN, Gohel S, Kelly C, Smith SM, et al. Toward discovery science of human brain function. Proc Natl Acad Sci U S A. 2010;107(10):4734-9. doi: 10.1073/pnas.0911855107.

[3] Fornito A, Zalesky A, Bullmore E. Fundamentals of brain network analysis. 1st ed. San Diego: Academic Press; 2016.

[4] Hagberg A, Swart P, S Chult D, editors. Exploring network structure, dynamics, and function using NetworkX. Proceedings of the 7th Python in Science conference (SciPy 2008); 2008 Aug 19-24; Pasadena, USA.

[5] Bassett DS, Sporns O. Network neuroscience. Nat Neurosci. 2017;20(3):353. doi: 10.1038/nn.4502.

[6] Santos FAN, Raposo EP, Coutinho-Filho MD, Copelli M, Stam CJ, Douw L. Topological phase transitions in functional brain networks. Phys Rev E. 2019;100(3-1):032414. doi: 10.1103/PhysRevE.100.032414.

<a id='acknowledgements'></a>

## <a id='toc4_3_'></a>[**6.** Acknowledgements](#toc0_)

This tutorial is adapted from "Centeno, E.G.Z., Moreni, G., Vriend, C. et al. A hands-on tutorial on network and topological neuroscience. Brain Struct Funct 227, 741–762 (2022)."
https://github.com/multinetlab-amsterdam/network_TDA_tutorial/tree/main.